In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
from scipy.spatial import distance
import pickle
import json
import h5py
import pandas as pd


cuda = torch.device('cuda')
frame_size = 10
# https://drive.google.com/open?id=1kTyu05ZmtP2MA33J5hWdX8OyUYEDW4iI
movie_ref = pickle.load(open('../data/infos_pca128.pytorch', 'rb'))
# on my system os error often pops up
# try to open the file explorer and copy paste the path again 
# https://drive.google.com/open?id=1pPf-7AmUVceVfgfmKEJ6ireEDKEJHw-7
f = h5py.File("../data/static_ml20m.hdf5", "r")
# https://drive.google.com/open?id=1xjVI4uVQGsQ7tjOJ3594ZXmAEC_6yX0e
meta = json.load(open('../data/metadata.json'))

In [3]:
class Actor(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=2e-1):
        super(Actor, self).__init__()
        
        self.drop_layer = nn.Dropout(p=0.5)
        
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, num_actions)
        
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        # state = self.state_rep(state)
        x = F.relu(self.linear1(state))
        x = self.drop_layer(x)
        x = F.relu(self.linear2(x))
        x = self.drop_layer(x)
        # x = torch.tanh(self.linear3(x)) # in case embeds are -1 1 normalized
        x = self.linear3(x) # in case embeds are standard scaled / wiped using PCA whitening
        # return state, x
        return x

In [4]:
ddpg = Actor(1290, 128, 256)
td3 = Actor(1290, 128, 256)
ddpg.load_state_dict(torch.load('../models/DDPG/ddpg_policy(Radam).pt'))
td3.load_state_dict(torch.load('../models/TD3/td3_policy_1.pt'))

batch_size = 5000
n_batches = (f['done'].shape[0] // batch_size) - 1

In [5]:
test_batch = [f[key][n_batches*batch_size:(n_batches+1)*batch_size] for key in
                 ['movies', 'ratings', 'done']]


movies, ratings, done = test_batch
movies, ratings, done = [torch.tensor(i.astype('float32')) for i in [movies, ratings, done]]
movies_tensor = torch.stack([torch.stack([movie_ref[int(i)] for i in ts]) for ts in movies])
    
state = torch.cat([movies_tensor[:, :-1, :].view(batch_size, -1),
                       ratings[:, :-1]], 1)


In [6]:
genres_dict = {'Animation': 16,
 'Comedy': 35,
 'Family': 10751,
 'Adventure': 12,
 'Fantasy': 14,
 'Romance': 10749,
 'Drama': 18,
 'Action': 28,
 'Crime': 80,
 'Thriller': 53,
 'Horror': 27,
 'History': 36,
 'Science Fiction': 878,
 'Mystery': 9648,
 'War': 10752,
 'Music': 10402,
 'Documentary': 99,
 'Western': 37,
 'TV Movie': 10770,
 'Sport': 1,
 'Sci-Fi': 878,
 'Musical': 10402,
 'Short': 2,
 'News': 3,
 'Film-Noir': 4}
genres_dict = {v: k for k, v in genres_dict.items()}

In [7]:
def rank(gen_action, metric):
    scores = []
    for i in movie_ref.keys():
        scores.append([i, metric(movie_ref[i], gen_action)])
    scores = list(sorted(scores, key = lambda x: x[1]))
    scores = scores[:10]
    ids = [i[0] for i in scores]
    for i in range(10):
        scores[i].extend([meta[str(scores[i][0])][key]  for key in ['title',
                                'genres', 'original_language', 'release_year', 'vote_average']])
        scores[i][3] = ' '.join([genres_dict[i] for i in scores[i][3]])

    indexes = ['id', 'score', 'title', 'genres', 'language', 'year', 'rating']
    table_dict = dict([(key,[i[idx] for i in scores]) for idx, key in enumerate(indexes)])
    table = pd.DataFrame(table_dict)
    return table

# DDPG

In [8]:
ddpg_action = ddpg(state)
# pick random action
ddpg_action = ddpg_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [9]:
from pandas.plotting import table
import subprocess
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(ddpg_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,31867,11.514376,Man of the House,Comedy Action,en,2005,5.4
1,4517,11.780486,Lady in White,Horror Thriller,en,1988,6.4
2,975,11.844872,Something to Sing About,Comedy,en,1937,5.4
3,2029,11.880260,Billy's Hollywood Screen Kiss,Comedy Romance,en,1998,6.1
4,84262,11.924979,Chain Letter,Horror Thriller,en,2010,4.1
5,92182,11.926578,"What's Up, Scarlet?",Comedy Romance,en,2005,5.0
6,4139,11.934120,No Man's Land,Action Drama Thriller Crime,en,1987,6.2
7,26402,12.003019,Harper Valley P.T.A.,Comedy,en,1978,7.0
8,6295,12.016489,Chasing Papi,Comedy Romance,en,2003,6.0
9,26755,12.023149,Stone Cold,Action Crime Thriller,en,1991,5.7


In [10]:
rank(ddpg_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,5751,0.680989,Goodbye Pork Pie,Adventure Comedy,en,1981,6.8
1,46372,0.689013,The Bribe,Crime Drama,en,1949,6.3
2,8825,0.694455,Cheerleader Camp,Comedy Horror Thriller,en,1988,5.0
3,116728,0.704141,Out of Bounds,Drama,da,2011,5.0
4,274,0.709339,Man of the House,Action Comedy Family,en,1995,5.1
5,7790,0.713101,"Bon Voyage, Charlie Brown (and Don't Come Back!)",Animation Comedy Family,en,1980,6.6
6,105815,0.713297,Undocumented,Horror Thriller,en,2010,6.5
7,27221,0.714524,Adrenaline Drive,Comedy Crime Romance,ja,1999,7.3
8,31867,0.714620,Man of the House,Comedy Action,en,2005,5.4
9,107647,0.715346,Wonderful and Loved by All,Comedy,sv,2007,6.0


In [11]:
rank(ddpg_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,46372,0.691771,The Bribe,Crime Drama,en,1949,6.3
1,27221,0.694565,Adrenaline Drive,Comedy Crime Romance,ja,1999,7.3
2,94486,0.705919,Ciao Bella,Drama Romance,en,2007,5.8
3,8825,0.706063,Cheerleader Camp,Comedy Horror Thriller,en,1988,5.0
4,5751,0.708943,Goodbye Pork Pie,Adventure Comedy,en,1981,6.8
5,1931,0.711389,Mutiny on the Bounty,Adventure Drama History,en,1935,7.4
6,92182,0.713215,"What's Up, Scarlet?",Comedy Romance,en,2005,5.0
7,3097,0.719133,The Shop Around the Corner,Comedy Romance Drama,en,1940,7.9
8,91799,0.726156,Skyscraper Souls,Drama Romance,en,1932,6.0
9,98348,0.726161,Watch the Birdie,Comedy Romance Crime,en,1950,5.7


In [12]:
rank(ddpg_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,57951,77.125975,Fool's Gold,Romance Comedy Adventure,en,2008,5.6
1,80903,77.221658,One Million B.C.,Fantasy Horror,en,1940,6.1
2,105111,77.474881,Bandit's Silver Angel,Comedy Action,en,1994,3.4
3,127313,78.139829,Evilenko,Drama Horror Thriller Crime,it,2004,5.7
4,76753,78.316071,The Final,Thriller Horror,en,2010,5.6
5,6250,78.362884,Dreamcatcher,Drama Horror Sci-Fi Thriller,en,2003,5.3
6,74727,78.740865,Gentlemen of Fortune,Comedy Crime Drama,ru,1971,7.6
7,106399,78.767434,The Wonderful Crook,Comedy Drama,fr,1974,5.6
8,128149,79.125798,Not Safe for Work,Thriller,en,2014,5.3
9,274,79.146993,Man of the House,Action Comedy Family,en,1995,5.1


In [13]:
rank(ddpg_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,31867,11.514376,Man of the House,Comedy Action,en,2005,5.4
1,4517,11.780486,Lady in White,Horror Thriller,en,1988,6.4
2,975,11.844872,Something to Sing About,Comedy,en,1937,5.4
3,2029,11.880260,Billy's Hollywood Screen Kiss,Comedy Romance,en,1998,6.1
4,84262,11.924979,Chain Letter,Horror Thriller,en,2010,4.1
5,92182,11.926578,"What's Up, Scarlet?",Comedy Romance,en,2005,5.0
6,4139,11.934120,No Man's Land,Action Drama Thriller Crime,en,1987,6.2
7,26402,12.003019,Harper Valley P.T.A.,Comedy,en,1978,7.0
8,6295,12.016489,Chasing Papi,Comedy Romance,en,2003,6.0
9,26755,12.023149,Stone Cold,Action Crime Thriller,en,1991,5.7


In [14]:
rank(ddpg_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,2029,2.283374,Billy's Hollywood Screen Kiss,Comedy Romance,en,1998,6.1
1,6831,2.463864,Nobody's Baby,Comedy,en,2001,5.1
2,1624,2.463892,A Thousand Acres,Drama,en,1997,5.9
3,122159,2.465633,The Affairs of Martha,Romance Comedy,en,1942,6.0
4,103923,2.480941,Aloha Summer,Comedy Drama,en,1988,5.0
5,5751,2.482299,Goodbye Pork Pie,Adventure Comedy,en,1981,6.8
6,6215,2.484798,Laurel Canyon,Drama,en,2002,6.0
7,4231,2.500946,Someone Like You...,Comedy Romance,en,2001,5.9
8,52,2.504043,Mighty Aphrodite,Comedy Romance,en,1995,6.8
9,3512,2.504220,Return to Me,Romance Comedy,en,2000,6.3


In [15]:
rank(ddpg_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,274,0.702106,Man of the House,Action Comedy Family,en,1995,5.1
1,4139,0.713185,No Man's Land,Action Drama Thriller Crime,en,1987,6.2
2,76753,0.713832,The Final,Thriller Horror,en,2010,5.6
3,7790,0.720075,"Bon Voyage, Charlie Brown (and Don't Come Back!)",Animation Comedy Family,en,1980,6.6
4,57951,0.720762,Fool's Gold,Romance Comedy Adventure,en,2008,5.6
5,89329,0.727456,Payment Deferred,Crime Thriller,en,1932,5.3
6,81067,0.728404,The Long and Short of It,,en,2003,6.6
7,40723,0.728795,Wolf Creek,Horror Thriller,en,2005,6.1
8,127313,0.729007,Evilenko,Drama Horror Thriller Crime,it,2004,5.7
9,87028,0.730646,The Roommate,Thriller Drama Horror,en,2011,5.3


In [16]:
rank(ddpg_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,4139,102.191505,No Man's Land,Action Drama Thriller Crime,en,1987,6.2
1,6295,104.591370,Chasing Papi,Comedy Romance,en,2003,6.0
2,31867,104.820946,Man of the House,Comedy Action,en,2005,5.4
3,4517,105.138176,Lady in White,Horror Thriller,en,1988,6.4
4,92182,106.184647,"What's Up, Scarlet?",Comedy Romance,en,2005,5.0
5,87028,106.241318,The Roommate,Thriller Drama Horror,en,2011,5.3
6,240,106.409210,Hideaway,Drama Thriller,en,1995,5.1
7,975,106.639862,Something to Sing About,Comedy,en,1937,5.4
8,1820,107.056290,The Proposition,Drama Romance,en,1998,5.7
9,105410,107.620285,Angel,Action Crime Thriller,en,1984,6.4


# TD3

In [17]:
td3_action = td3(state)
# pick random action
td3_action = td3_action[np.random.randint(0, 5000, 1)[0]].detach().cpu().numpy()

In [18]:
from pandas.plotting import table 
import matplotlib.pyplot as plt
%matplotlib inline

#from jupyterthemes import jtplot
#jtplot.style(theme='grade3')

rank(td3_action, distance.euclidean)

,id,score,title,genres,language,year,rating
0,69842,8.792418,Good Dick,Comedy Drama Romance,en,2008,6.2
1,7567,8.926256,Chance,Comedy Drama,en,2002,6.4
2,5451,8.976493,Pumpkin,Comedy Drama Romance,en,2002,5.9
3,8758,9.016467,Kotch,Comedy Drama,en,1971,6.8
4,5015,9.028862,Monster's Ball,Drama Romance,en,2001,6.7
5,72153,9.040674,A Dangerous Woman,Drama Romance,en,1993,5.2
6,67618,9.052890,Strictly Sexual,Drama Comedy Romance,en,2008,6.3
7,95967,9.072983,Marriage Retreat,Comedy Drama Romance,en,2011,6.1
8,83920,9.082467,Pete 'n' Tillie,Drama Comedy Romance,en,1972,6.1
9,48458,9.123867,St. Martin's Lane,Comedy Drama,en,1938,7.4


In [19]:
rank(td3_action, distance.cosine)

,id,score,title,genres,language,year,rating
0,5662,0.621782,The Wrong Guy,Action Comedy,en,1997,7.3
1,82368,0.665302,Alabama's Ghost,Horror Comedy,en,1973,4.0
2,105015,0.669000,The Last Run,Comedy Drama Romance,en,2004,4.2
3,55178,0.675032,Stephanie Daley,Drama,en,2006,6.0
4,104423,0.675897,The Snows of Kilimanjaro,Comedy Drama,fr,2011,6.4
5,2680,0.677305,Floating,Drama,en,1997,5.0
6,1648,0.680465,The House of Yes,Comedy Drama,en,1997,6.6
7,128665,0.685487,The Shepherd,Action Sci-Fi,en,1999,4.5
8,3008,0.687511,Last Night,Comedy Romance Drama,en,1998,7.0
9,88752,0.687532,Hide-Out,Comedy Crime Drama Romance,en,1934,7.4


In [20]:
rank(td3_action, distance.correlation) # looks similar to cosine

,id,score,title,genres,language,year,rating
0,5662,0.623257,The Wrong Guy,Action Comedy,en,1997,7.3
1,82368,0.666618,Alabama's Ghost,Horror Comedy,en,1973,4.0
2,105015,0.670264,The Last Run,Comedy Drama Romance,en,2004,4.2
3,2680,0.675736,Floating,Drama,en,1997,5.0
4,55178,0.676518,Stephanie Daley,Drama,en,2006,6.0
5,104423,0.677174,The Snows of Kilimanjaro,Comedy Drama,fr,2011,6.4
6,1648,0.680979,The House of Yes,Comedy Drama,en,1997,6.6
7,127080,0.685200,The Diary of Preston Plummer,Drama Romance,en,2012,4.7
8,128665,0.686359,The Shepherd,Action Sci-Fi,en,1999,4.5
9,88752,0.687720,Hide-Out,Comedy Crime Drama Romance,en,1934,7.4


In [21]:
rank(td3_action, distance.canberra)

,id,score,title,genres,language,year,rating
0,98473,78.313753,The Sleeping Car Murder,Crime,fr,1965,6.7
1,82015,78.793594,M,Drama Thriller,en,1951,6.3
2,55737,78.966870,Flight of the Living Dead,Horror Action,de,2007,5.1
3,5931,79.866882,Britannia Hospital,Comedy Drama Horror Sci-Fi,en,1982,7.1
4,7246,79.928639,The Violent Years,Drama Thriller,en,1956,3.9
5,73027,80.230816,The Last Station,Drama Romance,en,2009,6.6
6,95016,80.270225,The Peach Thief,Drama War Romance,bg,1964,7.1
7,100503,80.348884,I Thank a Fool,Drama,en,1962,6.0
8,56945,80.382552,The Perfect Holiday,Comedy Drama Romance Family,en,2007,5.4
9,8293,80.474153,Used People,Comedy Drama Romance,en,1992,6.0


In [22]:
rank(td3_action, distance.minkowski)

,id,score,title,genres,language,year,rating
0,69842,8.792418,Good Dick,Comedy Drama Romance,en,2008,6.2
1,7567,8.926256,Chance,Comedy Drama,en,2002,6.4
2,5451,8.976493,Pumpkin,Comedy Drama Romance,en,2002,5.9
3,8758,9.016467,Kotch,Comedy Drama,en,1971,6.8
4,5015,9.028862,Monster's Ball,Drama Romance,en,2001,6.7
5,72153,9.040674,A Dangerous Woman,Drama Romance,en,1993,5.2
6,67618,9.052890,Strictly Sexual,Drama Comedy Romance,en,2008,6.3
7,95967,9.072983,Marriage Retreat,Comedy Drama Romance,en,2011,6.1
8,83920,9.082467,Pete 'n' Tillie,Drama Comedy Romance,en,1972,6.1
9,48458,9.123867,St. Martin's Lane,Comedy Drama,en,1938,7.4


In [23]:
rank(td3_action, distance.chebyshev)

,id,score,title,genres,language,year,rating
0,68444,1.828049,The Great Buck Howard,Comedy,en,2008,5.9
1,1747,1.839261,Wag the Dog,Comedy Drama,en,1997,6.9
2,3337,1.871397,I'll Never Forget What's'isname,Comedy Drama,en,1967,6.8
3,2552,1.871658,My Boyfriend's Back,Comedy Romance,en,1993,5.3
4,117517,1.889544,Listen Up Philip,Comedy,en,2014,6.0
5,67618,1.906651,Strictly Sexual,Drama Comedy Romance,en,2008,6.3
6,4432,1.913471,Sweet Smell of Success,Drama,en,1957,7.7
7,34216,1.938404,Rick,Comedy Drama,en,2003,5.4
8,88823,1.940131,High Time,Comedy,en,1960,5.5
9,53847,1.944053,The Woman Chaser,Drama,en,1999,5.1


In [24]:
rank(td3_action, distance.braycurtis)

,id,score,title,genres,language,year,rating
0,5931,0.716681,Britannia Hospital,Comedy Drama Horror Sci-Fi,en,1982,7.1
1,55178,0.722164,Stephanie Daley,Drama,en,2006,6.0
2,5662,0.724294,The Wrong Guy,Action Comedy,en,1997,7.3
3,82015,0.726085,M,Drama Thriller,en,1951,6.3
4,98473,0.726972,The Sleeping Car Murder,Crime,fr,1965,6.7
5,117640,0.728874,Dr. Gillespie's New Assistant,Comedy Drama Mystery,en,1942,6.0
6,27509,0.730387,Carolina,Comedy Romance,en,2003,5.4
7,104423,0.730477,The Snows of Kilimanjaro,Comedy Drama,fr,2011,6.4
8,82368,0.734433,Alabama's Ghost,Horror Comedy,en,1973,4.0
9,3008,0.734932,Last Night,Comedy Romance Drama,en,1998,7.0


In [25]:
rank(td3_action, distance.cityblock)

,id,score,title,genres,language,year,rating
0,69842,76.731285,Good Dick,Comedy Drama Romance,en,2008,6.2
1,2257,78.646835,No Small Affair,Comedy Drama Romance,en,1984,5.6
2,7567,78.730911,Chance,Comedy Drama,en,2002,6.4
3,8758,79.747787,Kotch,Comedy Drama,en,1971,6.8
4,83920,79.906288,Pete 'n' Tillie,Drama Comedy Romance,en,1972,6.1
5,5015,80.051659,Monster's Ball,Drama Romance,en,2001,6.7
6,72,80.736488,Kicking and Screaming,Comedy Drama Romance,en,1995,6.9
7,121821,80.885201,The Dancer,Drama,id,2011,5.9
8,95967,81.061584,Marriage Retreat,Comedy Drama Romance,en,2011,6.1
9,72153,81.124878,A Dangerous Woman,Drama Romance,en,1993,5.2
